We are going to build two types of content-based recommender:

Plot description-based recommender: This model compares the descriptions and taglines of different movies, and provides recommendations that have the most similar plot descriptions.

In [1]:
import pandas as pd
import numpy as np

In [2]:

data = pd.read_csv("./data/movies_metadata.csv")

/home/arun/Documents/animerec/animerecommendation/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [4]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [7]:
df = data[['title', 
       'vote_average', 'genres', 'overview','id','vote_count']]


In [8]:
df

,title,vote_average,genres,overview,id,vote_count
0,Toy Story,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",862,5415.0
1,Jumanji,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,8844,2413.0
2,Grumpier Old Men,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,15602,92.0
3,Waiting to Exhale,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",31357,34.0
4,Father of the Bride Part II,5.7,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,11862,173.0
...,...,...,...,...,...,...
45461,Subdue,4.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",Rising and falling between a man and woman.,439050,1.0
45462,Century of Birthing,9.0,"[{'id': 18, 'name': 'Drama'}]",An artist struggles to finish his work while a...,111109,3.0
45463,Betrayal,3.8,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","When one of her hits goes wrong, a professiona...",67758,6.0
45464,Satan Triumphant,0.0,[],"In a small town live two brothers, one a minis...",227506,0.0


In [9]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['overview'] = df['overview'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix


/tmp/ipykernel_210266/3565993951.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overview'] = df['overview'].fillna('')


<45466x75827 sparse matrix of type '<class 'numpy.float64'>'
	with 1210882 stored elements in Compressed Sparse Row format>

In [10]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

In [11]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [12]:
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64

In [19]:
idx = indices['Betrayal']

In [20]:
idx

title
Betrayal    15294
Betrayal    45463
dtype: int64

In [21]:
sim_scores = list(enumerate(linear_kernel(tfidf_matrix[15294], tfidf_matrix).flatten()))

In [22]:
sim_scores

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.005105431857788875),
 (21, 0.0),
 (22, 0.0),
 (23, 0.02683805253345206),
 (24, 0.0),
 (25, 0.038349038912442075),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.01020074673669222),
 (35, 0.020377680730975328),
 (36, 0.0),
 (37, 0.016417020156532226),
 (38, 0.0),
 (39, 0.0),
 (40, 0.03925313421339245),
 (41, 0.0),
 (42, 0.016395225187116943),
 (43, 0.0),
 (44, 0.03362118800475521),
 (45, 0.008763992173147484),
 (46, 0.0),
 (47, 0.022612927185676153),
 (48, 0.020968908847547853),
 (49, 0.005667869177529908),
 (50, 0.01662374461733197),
 (51, 0.0),
 (52, 0.0),
 (53, 0.0),
 (54, 0.024515856953496784),
 (55, 0.0),
 (56, 0.0),
 (57, 0.0),
 (58, 0.0),
 (59, 0.0),
 (60, 0.0),
 (61, 0.0),
 (62, 0.0),
 (63

In [23]:
    # Sort the movies based on the cosine similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
sim_scores = sim_scores[1:11]

In [24]:
sim_scores

[(31255, 0.24072796751601153),
 (25009, 0.14385256658888698),
 (21348, 0.14347985570756588),
 (4305, 0.14032652012530006),
 (347, 0.13864805010114856),
 (34027, 0.1378626710920542),
 (8479, 0.1364297805323325),
 (13401, 0.13407592279714897),
 (7425, 0.13354963524153046),
 (43865, 0.13161139368025981)]

In [26]:
movie_indices = [i[0] for i in sim_scores]
df['title'].iloc[movie_indices]

31255                Post Coitum
25009         The New Girlfriend
21348          Darling Companion
4305         Written on the Wind
347                     Crooklyn
34027        The Perfect Husband
8479              Clash by Night
13401    Fireflies in the Garden
7425                Undercurrent
43865             Reverse Runner
Name: title, dtype: object

In [13]:
# Function that takes in movie title as input and gives recommendations 
def content_recommender(title, tfidf_matrix=tfidf_matrix, df=df, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(linear_kernel(tfidf_matrix[idx], tfidf_matrix).flatten()))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [17]:
#Get recommendations for The Lion King
content_recommender('Betrayal')

IndexError: positional indexers are out-of-bounds

In [2]:
import pandas as pd
ratings="https://github.com/couturierc/tutorials/raw/master/recommender_system/data/ratings.csv"
movies="https://github.com/couturierc/tutorials/raw/master/recommender_system/data/movies.csv"

# If data stored locally
# ratings="./data/ratings.csv"
# movies="./data/movies.csv"

df_ratings = pd.read_csv(ratings, sep=',')
df_ratings.columns = ['userId', 'itemId', 'rating', 'timestamp']
df_movies = pd.read_csv(movies, sep=',')
df_movies.columns = ['itemId', 'title', 'genres']

In [3]:
df_ratings

,userId,itemId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [2]:
!pip install surprise 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
ratings="https://github.com/couturierc/tutorials/raw/master/recommender_system/data/ratings.csv"
movies="https://github.com/couturierc/tutorials/raw/master/recommender_system/data/movies.csv"

# If data stored locally
# ratings="./data/ratings.csv"
# movies="./data/movies.csv"

df_ratings = pd.read_csv(ratings, sep=',')
df_ratings.columns = ['userId', 'itemId', 'rating', 'timestamp']
df_movies = pd.read_csv(movies, sep=',')
df_movies.columns = ['itemId', 'title', 'genres']


In [7]:
# from surprise import Reader, Dataset, SVD, evaluate

# Following Surprise documentation examples 
# https://surprise.readthedocs.io/en/stable/getting_started.html evaluate, NormalPredictor

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from collections import defaultdict

# As we're loading a custom dataset, we need to define a reader.
reader = Reader(rating_scale=(0.5, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_ratings[['userId', 'itemId', 'rating']], reader)

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8736  0.8730  0.8702  0.8710  0.8789  0.8733  0.0031  
MAE (testset)     0.6706  0.6725  0.6703  0.6674  0.6763  0.6714  0.0029  
Fit time          2.95    2.96    2.91    2.92    2.91    2.93    0.02    
Test time         0.12    0.14    0.09    0.09    0.14    0.12    0.02    


{'test_rmse': array([0.87359964, 0.8729894 , 0.87017126, 0.87104097, 0.87891535]),
 'test_mae': array([0.67055768, 0.67249144, 0.6703488 , 0.66741716, 0.67633543]),
 'fit_time': (2.9508724212646484,
  2.9647231101989746,
  2.9084620475769043,
  2.9208695888519287,
  2.9073338508605957),
 'test_time': (0.11757278442382812,
  0.13939952850341797,
  0.08919525146484375,
  0.08649516105651855,
  0.14271831512451172)}

In [8]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.8940260233556172
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [13]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
trainset = data.build_full_trainset()
algo.fit(trainset)

In [14]:
algo.predict(621,1)

Prediction(uid=621, iid=1, r_ui=None, est=3.782138553409833, details={'was_impossible': False})

In [10]:
df_data = data.df
df_data = df_data.join(df_movies,how="left", on='itemId',rsuffix='_', lsuffix='')
df_data[df_data['userId']==1].sort_values(by = 'rating',ascending=False)[:10]

,userId,itemId,rating,itemId_,title,genres
231,1,5060,5.0,7932.0,Dark Days (2000),Documentary
185,1,2872,5.0,3840.0,Pumpkinhead (1988),Horror
89,1,1291,5.0,1721.0,Titanic (1997),Drama|Romance
90,1,1298,5.0,1732.0,"Big Lebowski, The (1998)",Comedy|Crime
190,1,2948,5.0,3952.0,"Contender, The (2000)",Drama|Thriller
189,1,2947,5.0,3951.0,Two Family House (2000),Drama
188,1,2944,5.0,3948.0,Meet the Parents (2000),Comedy
186,1,2899,5.0,3882.0,Bring It On (2000),Comedy
184,1,2858,5.0,3824.0,Autumn in New York (2000),Drama|Romance
179,1,2700,5.0,3623.0,Mission: Impossible II (2000),Action|Adventure|Thriller


In [15]:
# From Surprise documentation: https://surprise.readthedocs.io/en/stable/FAQ.html
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
# Predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)